# 智慧運算技術導論 (Week 4 自然語言處理主題 - Day4)

## 作業：RAG 系統
- 使用資料集：[PubMedQA (pqa_labeled)](https://huggingface.co/datasets/qiaojin/PubMedQA/viewer/pqa_labeled)
- 你的任務：完成所有的 `TODO`（一個 # Write your code here 代表有一行程式碼要寫）
- 如果有任何問題歡迎舉手詢問老師
- 雖然有些格子無需修改，但仍建議理解程式碼

## 分數判定
- 完成所有 `TODO`: `60%`，任 1 `TODO` 為 `6分`。
- 最後一格準確率達到 50% 以上: `10%`
- 報告繳交: `30%`，內容需涵蓋：
  - 請說明你所使用的 prompts
  - 請說明本次作業的方法與你所使用的模型
  - 請分享你遇到的問題或你覺得很困難的地方
  - (如果有的話) 請說明你如何用 AI 完成此份作業
- 繳交方式：
  1. 準備一個資料夾
  2. 放進去此檔案跟報告pdf
  3. 將此資料夾壓縮為 `學號.zip`
  4. 上傳至 Teams 作業區

### 安裝與引入套件

In [ ]:
!pip install 'aisuite[all]' gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of mistralai to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.6 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of mistralai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB

In [ ]:
# ==================== 0. 引入套件 ====================

import os
import json
import gradio as gr
import aisuite as ai
from google.colab import userdata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# ==================== 0. 模型與資料設定 ====================

MODEL_NAME = "groq:openai/gpt-oss-120b" # 可以換成你喜歡的模型
NUM_TEST_SAMPLES = 10                   # 請勿修改此數字，我們只測 10 筆就好

In [ ]:
# ==================== 1. 載入資料（此格無需修改）====================

print("載入 PubMedQA 資料...")
with open("pubmedqa_100.json", 'r', encoding="utf-8") as f:
    test_data = json.load(f)

with open("pubmedqa_train_corpus.json", 'r', encoding="utf-8") as f:
    all_data = json.load(f)

print(f"✅ 載入完成！共 {len(all_data)} 筆語料庫（Corpus）")
print(f"✅ 載入完成！共 {len(test_data)} 筆醫學問答")
print("\n範例資料：")
print(f"問題: {test_data[0]['question']}")
print(f"答案: {test_data[0]['answer']}")
print()

載入 PubMedQA 資料...
✅ 載入完成！共 1000 筆語料庫（Corpus）
✅ 載入完成！共 100 筆醫學問答

範例資料：
問題: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?
答案: yes



In [ ]:
# ==================== 2. 設定 Groq API ====================

# TODO1: 請註冊 Groq 帳號，並填入自己的 API_KEY
os.environ['GROQ_API_KEY'] = "YOUR_API_KEY"
# 或是使用 Colab Secret Key 方式
# os.environ['GROQ_API_KEY'] = userdata.get('groq')

client = ai.Client()
print("✅ Groq 設定完成")

✅ Groq 設定完成


### TF-IDF

In [ ]:
# ==================== 3. 建立 TF-IDF 檢索器 ====================

corpus_train = [item['context'] for item in all_data]
corpus_test = [item['context'] for item in test_data]

# TODO2: 請創建 TF-IDF 檢索器變數
vectorizer = # Write your code here

# TODO3: 請建立 TF-IDF 檢索器
# Write your code here
tfidf_matrix = vectorizer.fit_transform(corpus_test)

print("✅ TF-IDF 檢索器建立完成")
print(f"   - 詞彙量: {len(vectorizer.get_feature_names_out())}")
print(f"   - 文檔數: {tfidf_matrix.shape[0]}")

✅ TF-IDF 檢索器建立完成
   - 詞彙量: 3517
   - 文檔數: 100


In [ ]:
# ==================== 4. 檢索函數（此格無需修改）====================

def retrieve_documents(retriever: TfidfVectorizer, query: str, top_k: int = 3):
    """使用 TF-IDF 檢索最相關的文檔"""

    # TODO4：請利用 retriever 對 query 進行向量轉換
    query_vec = # Write your code here

    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]

    # 整理檢索結果
    results = []
    for idx in top_indices:
        results.append({
            "rank": len(results) + 1,
            "similarity": float(similarities[idx]),
            "context": test_data[idx]['context'],
            "question": test_data[idx]['question']
        })

    return results

### RAG 系統

In [ ]:
# ==================== 5. LLM 生成函數 ====================

def generate_answer(model: str, question: str, retrieved_docs: list) -> str:
    """使用 Groq LLM 根據檢索到的文檔生成答案"""

    # TODO5: 請針對參考文件區塊，設計你的 prompt
    context = ""
    for doc in retrieved_docs:
        context += f"{doc['context']}\n\n"  # Write your code here（對此行進行修改）

    # TODO6: 請設計你的 prompt，讓模型知道參考文件區塊的提示
    # TODO7: 請修改 Question: 那行以下的 prompt，讓模型知道它應該回答什麼
    prompt = f"""Based on the following medical literature ... # Write your code here（對此行進行修改）

Medical Literature:
{context}

Question: {question}
                                          # Write your code here（你不一定要換行）
Format:                                   # Write your code here（對此行進行修改）
1. First answer 'yes', 'no', or 'maybe'.  # Write your code here（對此行進行修改）
2. Then                                   # Write your code here（對此行進行修改）
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            # TODO8: 請給予模型 system prompt
            # Write your code here
            {"role": "user", "content": prompt}
        ],
        # TODO9: 請設定最長 tokens 數目
        max_tokens=, # Write your code here
        # TODO10: 請設定溫度係數
        temperature=, # Write your code here
    )

    return response.choices[0].message.content

In [ ]:
# ==================== 6. 完整的 RAG 流程（此格無需修改） ====================

def rag_system(retriever: TfidfVectorizer, model: str, question: str, top_k: int = 3) -> str:
    """完整的 RAG 系統：檢索 + 生成"""

    print(f"\n🔍 正在檢索相關文獻...")

    # Step 1: 檢索
    retrieved_docs = retrieve_documents(retriever, question, top_k=top_k)

    print(f"✅ 找到 {len(retrieved_docs)} 篇相關文獻")
    for doc in retrieved_docs:
        print(f"  - 相似度 {doc['similarity']:.3f}: {doc['context'][:80]}...")

    # Step 2: 生成
    print(f"\n🤖 正在生成答案...")
    answer = generate_answer(model, question, retrieved_docs)

    return answer

In [ ]:
# ==================== 7. 測試單一問題（此格無需修改） ====================

print("\n" + "="*60)
print("測試 RAG 系統")
print("="*60)

test_question = test_data[0]['question']
print(f"\n問題: {test_question}")

answer = rag_system(
    retriever=vectorizer,
    model=MODEL_NAME,
    question=test_question,
    top_k=3
)

print(f"\n💡 RAG 系統的答案：")
print(answer)
print(f"\n📚 標準答案：{test_data[0]['answer']}")


測試 RAG 系統

問題: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?

🔍 正在檢索相關文獻...
✅ 找到 3 篇相關文獻
  - 相似度 0.276: Programmed cell death (PCD) is the regulated death of cells within an organism. ...
  - 相似度 0.081: The endogenous estradiol metabolite, 2-methoxyestradiol (2ME), has been shown to...
  - 相似度 0.070: Using murine models, we have shown that the lysosomotropic amine, chloroquine, i...

🤖 正在生成答案...

💡 RAG 系統的答案：
maybe  
The excerpt from Document 1 describes the spatial pattern of PCD in lace‑plant leaf areoles but does not mention mitochondria or their involvement in the remodeling process. While mitochondria are known to participate in many plant programmed cell‑death pathways, the provided literature gives no direct evidence for or against their role in lace‑plant leaf perforation, leaving the answer uncertain.

📚 標準答案：yes


In [ ]:
# ==================== 8. 取得模型對於前10筆資料的預測結果（此格無需修改） ====================

predictions = []

for i in range(NUM_TEST_SAMPLES):
    correct = 0
    question = test_data[i]['question']
    ground_truth = test_data[i]['answer'].lower()
    context = test_data[i]['context']

    answer = rag_system(
        retriever=vectorizer,
        model=MODEL_NAME,
        question=test_question,
        top_k=3
    )
    predictions.append(answer)


🔍 正在檢索相關文獻...
✅ 找到 3 篇相關文獻
  - 相似度 0.276: Programmed cell death (PCD) is the regulated death of cells within an organism. ...
  - 相似度 0.081: The endogenous estradiol metabolite, 2-methoxyestradiol (2ME), has been shown to...
  - 相似度 0.070: Using murine models, we have shown that the lysosomotropic amine, chloroquine, i...

🤖 正在生成答案...

🔍 正在檢索相關文獻...
✅ 找到 3 篇相關文獻
  - 相似度 0.276: Programmed cell death (PCD) is the regulated death of cells within an organism. ...
  - 相似度 0.081: The endogenous estradiol metabolite, 2-methoxyestradiol (2ME), has been shown to...
  - 相似度 0.070: Using murine models, we have shown that the lysosomotropic amine, chloroquine, i...

🤖 正在生成答案...

🔍 正在檢索相關文獻...
✅ 找到 3 篇相關文獻
  - 相似度 0.276: Programmed cell death (PCD) is the regulated death of cells within an organism. ...
  - 相似度 0.081: The endogenous estradiol metabolite, 2-methoxyestradiol (2ME), has been shown to...
  - 相似度 0.070: Using murine models, we have shown that the lysosomotropic amine, chloroquine, i...

In [ ]:
# ==================== 9. 準確率評估函數（此格無需修改） ====================

def calculate_accuracy(predictions: list, ground_truths: list) -> float:
    """計算 RAG 系統的準確率"""

    correct = 0

    print(f"\n{'='*60}")
    print(f"開始評估 {NUM_TEST_SAMPLES} 個問題...")
    print('='*60)

    for i, ground_truth in enumerate(ground_truths):
        try:
            answer = predictions[i]
            answer_lower = answer.lower()

            if ground_truth in answer_lower:
                correct += 1
                print(f"✅ 問題 {i+1}: 正確")
            else:
                print(f"❌ 問題 {i+1}: 錯誤 (標準答案: {ground_truth})")
                print(f"   系統預測: {answer}")

        except Exception as e:
            print(f"⚠️ 問題 {i+1}: 發生錯誤 - {e}")

    accuracy = (correct / NUM_TEST_SAMPLES) * 100

    print(f"\n{'='*60}")
    print(f"準確率: {correct}/{NUM_TEST_SAMPLES} = {accuracy:.1f}%")
    print('='*60)

    return accuracy

In [ ]:
# ==================== 10. 執行模型評估 ====================
# 此格理論上無需修改，但前提是模型預測結果之變數名稱需為 `predictions`

ground_truths = [item['answer'].lower() for item in test_data[:NUM_TEST_SAMPLES]]
accuracy = calculate_accuracy(predictions, ground_truths)


開始評估 10 個問題...
❌ 問題 1: 錯誤 (標準答案: yes)
   系統預測: maybe  
The excerpt about lace plant leaf perforation describes where PCD occurs but does not mention mitochondria. While mitochondria are commonly involved in programmed cell death signaling in plants, the provided literature does not give direct evidence for their role in lace‑plant leaf remodeling. Hence, based on the available documents the answer is uncertain.
✅ 問題 2: 正確
❌ 問題 3: 錯誤 (標準答案: yes)
   系統預測: maybe
The excerpt from Document 1 describes the spatial pattern of programmed cell death in lace plant leaves but does not mention mitochondria or any mitochondrial involvement in the leaf‑remodelling process. Since the provided literature gives no evidence either way, we cannot confirm a role for mitochondria.
✅ 問題 4: 正確
❌ 問題 5: 錯誤 (標準答案: yes)
   系統預測: maybe  
The provided excerpt about lace plant leaf perforation describes where PCD occurs but does not mention mitochondria or their involvement. While mitochondria are commonly implica